In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

#import necessary libraries.
import tensorflow as tf
import numpy as np
layer = tf.keras.layers

print('check tensorflow version : ', tf.__version__)

# tf.data API

tf.data API는 데이터셋을 모델에 연결해주기 위한 복합적인 입력 파이프라인을 구축할 수 있게 도와줍니다.

다수의 분산된 파일로부터 통합된 데이터를 만들어야 하는 경우나, 데이터 전처리, 미니배치, 랜덤셔플링 등의 데이터 파이프라인을 위한 복잡한 구조를 높은 추상성으로 간단하게 제어할 수 있는 인터페이스를 제공합니다.

https://www.tensorflow.org/guide/data

### 1. from_tensor_slices : python으로부터 데이터 받기

In [ ]:
data = np.array(list(range(100)), np.float32) #파이썬 데이터 0,1,2,3,4,...99,100
print(data[:10])

your_first_dataset = tf.data.Dataset.from_tensor_slices(data) #텐서플로우 데이터셋 오브젝트로 읽기

In [ ]:
# 데이터셋 오브젝트는 for 문에 의해 iteration 가능함.
for elm in your_first_dataset:
    print(elm)

In [ ]:
# 정상적으로 데이터를 가져오는 지 보기 위해 한 개의 데이터를 보고 싶을 수도 있습니다.
# 첫 번째 방법(take함수를 사용해서)
for element in your_first_dataset.take(1):
    print(element)
# 두 번째 방법
print(next(iter(your_first_dataset)))

In [ ]:
#데이터 개별적으로 처리하기 위한 함수를 정의해둡니다. 이때 함수는 tensorflow의 연산자 또는 함수로 정의된 것을 사용해야 합니다.
def multiply(element):
    multiplied = 10*element
    return multiplied

def add(element):
    added = element + 0.5
    return added

#데이터셋을 파이프라인으로 가져오는 중에 사전 정의한 함수에 의해 처리될 수 있습니다.
#데이터셋 ----> map(multiply) ----> map(add) ----> batch ----> ....

print("데이터를 map 함수를 통해 병렬적으로 처리하도록 할 수 있습니다.")
your_pipeline1 = your_first_dataset.map(multiply)
for elm in your_pipeline1.take(5):
    print(elm)
print("또한 연속적으로 데이터를 처리할 수도 있습니다.")
your_pipeline2 = your_first_dataset.map(multiply).map(add)
for elm in your_pipeline2.take(5):
    print(elm)

In [ ]:
# 파이프라인은 다음과 같이 합쳐질 수도 있습니다.
new_pipeline = tf.data.Dataset.zip((your_pipeline1, your_pipeline2))
#데이터셋1 ----> map(multiply) -------------------> | 
#                                                 |(zip)----> new_pipeline
#데이터셋2 ----> map(multiply) ----> map(add) ----> |
for elm1, elm2 in new_pipeline.take(5):
    print(elm1, elm2)
    

In [ ]:
# 전처리 구간을 지나서 자주 사용하는 파이프라인의 기능은 
# cache(데이터 일부를 캐시메모리에 저장할 수 있도록 함), shuffle(데이터셋을 shuffling해서 가져옴)
# batch(mini-batch로 데이터 가져옴), prefetch(모델이 s번째 데이터를 읽는 동안, s+1번째 데이터를 준비)
M = 100 # space for shuffle
N = 8 # batch size
new_pipeline = new_pipeline.cache().shuffle(M).batch(N).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for elm1, elm2 in new_pipeline.take(1):
    print(elm1, elm2)

# 1. 메모리 공간 위 이미지를 가져오는 경우

In [ ]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
images, labels = train
images = images/255

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset

# 2. TextLineDataset : text file로부터 데이터셋 생성

In [ ]:
directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']

file_paths = [
    tf.keras.utils.get_file(file_name, directory_url+file_name)
    for file_name in file_names ]

In [ ]:
print(file_paths)

In [ ]:
dataset = tf.data.TextLineDataset(file_paths)

In [ ]:
for line in dataset.take(1):
  print(line.numpy())

In [ ]:
new_dataset = dataset.shuffle(buffer_size=10000).batch(3)

In [ ]:
for line in new_dataset.take(1):
    print(line.numpy())

# 3. TextLineDataset : csv 파일로부터 읽어오기

In [ ]:
import pandas as pd
titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
df = pd.read_csv(titanic_file, index_col=None)
df.head()

In [ ]:
titanic_slices = tf.data.Dataset.from_tensor_slices(dict(df))

for feature_batch in titanic_slices.take(1):
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))